# Clustering of Sentinel-2 Timeseries Zonal Mean

In this notebook, you will conduct a kernel k means clustering algorithm on a time series of satellite remote sensing data. The time series data can be zonal statistics from OpenET ET rasters or Sentinel-2 vegetation index data. The zones for calculating the zonal statistics are determined by the LandIQ field polygons.

This notebook uses the tslearn package for doing the kernel k means clustering. See the tslearn documentation for more details: https://tslearn.readthedocs.io/en/stable/index.html

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import curve_fit
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import KernelKMeans

ModuleNotFoundError: No module named 'geopandas'

The following section shows the implementation of the time series clustering on zonal stats satellite remote sensing data in object-oriented programming style. The class TimeSeries Analysis line defines an instance of TimeSeries Analysis that conveys the user-selected parameters for each run of the time series clustering algorithm.

In [ ]:
class TimeSeriesAnalysis:
    def __init__(self, timeseries, landiq, sentinel, class_columns, subclass_columns, class_values, subclass_values, startdate, enddate, varColumn, idColumn, dateColumn):
        self.timeseries = timeseries
        self.landiq = landiq
        self.sentinel = sentinel
        self.class_columns = class_columns
        self.subclass_columns = subclass_columns
        self.class_values = class_values
        self.subclass_values = subclass_values
        self.startdate = startdate
        self.enddate = enddate
        self.varColumn = varColumn
        self.idColumn = idColumn
        self.dateColumn = dateColumn
        self.ids = None
        self.reshaped = None
        self.clusters = None
        self.pivoted = None
        self.Xtrain = None
        self.results = None

    def reshaping(self, cloudydates):
        
        # Filter the LandIQ data by the specified crop class 
        class_conditions = []
        for column in self.class_columns:
            class_conditions.append(self.landiq[column].isin(self.class_values))
        combined_class_condition = pd.concat(class_conditions, axis=1).any(axis=1)    
        if self.subclass_values:
            subclass_conditions = []
            for column in self.subclass_columns:
                subclass_conditions.append(self.landiq[column].isin(self.subclass_values))
            combined_subclass_condition = pd.concat(subclass_conditions, axis=1).any(axis=1)
            self.landiq = self.landiq[combined_class_condition & combined_subclass_condition]
        else:
            self.landiq = self.landiq[combined_class_condition]
            
        # Find the unique LandIQ field IDs for the specified crop class
        self.ids = pd.unique(self.landiq[self.idColumn])

        # Convert the date column to datetime in timeseries data
        self.timeseries[self.dateColumn] = pd.to_datetime(self.timeseries[self.dateColumn])
        
        # Filter the timeseries by dates
        self.timeseries = self.timeseries[(self.timeseries[self.dateColumn] >= self.startdate) & (self.timeseries[self.dateColumn] <= self.enddate)]
       
        # Filter the timeseries by the LandIQ field IDs for the specified crop class
        self.timeseries = self.timeseries[self.timeseries[self.idColumn].isin(self.ids)]

        # Reshape for clustering
        self.timeseries[self.dateColumn] = pd.to_datetime(self.timeseries[self.dateColumn])
        self.timeseries.sort_values(self.dateColumn)
        self.pivoted = pd.pivot_table(self.timeseries, index=self.idColumn, columns=self.dateColumn, values=self.varColumn, aggfunc=list)
        self.pivoted.dropna(inplace=True)
        ind = self.pivoted.index
        self.reshaped = np.array(self.pivoted.values.tolist())
        ind = np.array(ind.values.tolist())

    def run_analysis(self, num_clusters):   

        # Filter the LandIQ data by the specified crop class 
        class_conditions = []
        for column in self.class_columns:
            class_conditions.append(self.landiq[column].isin(self.class_values))
        combined_class_condition = pd.concat(class_conditions, axis=1).any(axis=1)    
        if self.subclass_values:
            subclass_conditions = []
            for column in self.subclass_columns:
                subclass_conditions.append(self.landiq[column].isin(self.subclass_values))
            combined_subclass_condition = pd.concat(subclass_conditions, axis=1).any(axis=1)
            self.landiq = self.landiq[combined_class_condition & combined_subclass_condition]
        else:
            self.landiq = self.landiq[combined_class_condition]
            
        # Find all the combinations of crop classes across the 4 growth stages
        class_combos = pd.unique(self.landiq['LIQ_REPORT'])
        mainclass_combos = pd.unique(self.landiq['MAIN_CROP'])

        # Find the unique LandIQ field IDs for the specified crop class
        self.ids = pd.unique(self.landiq[self.idColumn])

        # Convert the date column to datetime in timeseries data
        self.timeseries[self.dateColumn] = pd.to_datetime(self.timeseries[self.dateColumn])
        
        # Filter the timeseries by dates and exclude additional dates
        self.timeseries = self.timeseries[(self.timeseries[self.dateColumn] >= self.startdate) & (self.timeseries[self.dateColumn] <= self.enddate)]
        
        # Filter the timeseries by the LandIQ field IDs for the specified crop class
        self.timeseries = self.timeseries[self.timeseries[self.idColumn].isin(self.ids)]
        
        # Reshape for clustering
        self.timeseries[self.dateColumn] = pd.to_datetime(self.timeseries[self.dateColumn])
        self.timeseries.sort_values(self.dateColumn)
        self.pivoted = pd.pivot_table(self.timeseries, index=self.idColumn, columns=self.dateColumn, values=self.varColumn, aggfunc=list)
        self.pivoted.dropna(inplace=True)
        ind = self.pivoted.index
        self.reshaped = np.array(self.pivoted.values.tolist())
        ind = np.array(ind.values.tolist())

        # Apply Kernel KMeans
        self.Xtrain = self.reshaped
        seed = 0
        np.random.seed(seed)
        permutation = np.random.permutation(len(self.Xtrain))
        self.Xtrain = self.Xtrain[permutation]
        ind = ind[permutation]
        self.Xtrain = TimeSeriesScalerMeanVariance().fit_transform(self.Xtrain)
        gak_km = KernelKMeans(n_clusters=num_clusters, kernel="gak", kernel_params={"sigma": "auto"}, n_init=20, verbose=True, random_state=seed)
        self.clusters = gak_km.fit_predict(self.Xtrain)
        self.results = pd.DataFrame({'UniqueID': ind, 'Cluster': self.clusters})
        self.results = self.results.merge(self.landiq, left_on='UniqueID', right_on='UniqueID')
    
    def run_mapping(self, features): # add self. where needed
        
        # Convert the data type of the common column in the pandas DataFrame
        features["UniqueID"] = features["UniqueID"].astype(self.results["UniqueID"].dtype)
        
        # Join the LandIQ geodataframe and with the clustering results
        joined = features.merge(self.results, on="UniqueID")
     
        # Plot the fields 
        fig, ax = plt.subplots()
        features.plot(ax=ax, color='grey', label='Other')
        unique_values = joined['Cluster'].unique()
        if num_clusters <= 5:
            colors = ['red', 'blue', 'green', 'orange', 'purple'][:num_clusters]
        else:
            colors = cm.get_cmap('Spectral', num_clusters)
        for value in unique_values:
            subset = joined[joined['Cluster'] == value]
            subset.plot(ax=ax, color=colors[value], label=value)  
        ax.set_title('Field Boundaries')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        #ax.grid(True)
        if subclass_values==['**']:
            filename = f"cluster_map_{class_columns}_{class_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
        else: 
            filename = f"cluster_map_{class_columns}_{class_values}_{subclass_columns}_{subclass_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
        plt.savefig(filename)        
        plt.show()
        
        return joined
    
    def run_graphing(self):
        
        # Days of the year
        date_object = datetime.datetime.strptime(self.startdate, '%Y-%m-%d')
        year = date_object.year
        if year == 2020:
            days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        else:
            days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        doy = np.cumsum(days_in_month)
        
        # Calculate the sum and mean of the ET data points in each cluster
        max_cluster = np.max(self.clusters)
        cluster_sums = []
        cluster_means = []
        for cluster in range(max_cluster + 1):
            cluster_sum = np.sum(self.reshaped[self.clusters == cluster], axis=(1, 2))
            cluster_mean = np.mean(cluster_sum)
            cluster_sums.append(cluster_sum)
            cluster_means.append(cluster_mean)
        print("Cluster mean:", cluster_means)
                 
        # Plotting each cluster's ET data points on top in different colors
        cluster_counts = [np.sum(self.clusters == cluster_idx) for cluster_idx in range(num_clusters)]
        sorted_clusters = np.argsort(cluster_counts)[::-1]
        fig, ax2 = plt.subplots()
        if num_clusters <= 5:
            colors = ['red', 'blue', 'green', 'orange', 'purple'][:num_clusters]
        else:
            colors = cm.get_cmap('Spectral', num_clusters)
        for cluster_idx in sorted_clusters:
            cluster_indices = np.where(self.clusters == cluster_idx)[0]
            for idx in cluster_indices:
                if varColumn=='et' or varColumn=='et_ensemble_mad':
                    ax2.plot(doy, self.reshaped[idx].squeeze(), color=colors[cluster_idx], linewidth=0.25)
                else: 
                    ax2.plot(self.timeseries[dateColumn].unique(), self.reshaped[idx].squeeze(), color=colors[cluster_idx], linewidth=0.25)
        months = range(1, 13)
        month_labels = [datetime.date(2000, month, 1).strftime('%b') for month in months]
        if varColumn=='et' or varColumn=='et_ensemble_mad':
            ax2.set_xticks(doy)
            ax2.set_xticklabels(month_labels)
        else: 
            ax2.set_xticklabels(self.timeseries[dateColumn].unique())    
        ax2.set_xlabel(dateColumn)            
        ax2.set_title("All Points")
        ax2.set_ylabel(varColumn)
        plt.tight_layout()
        if subclass_values==['**']:
            filename = f"cluster_plot_{class_columns}_{class_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
        else: 
            filename = f"cluster_plot_{class_columns}_{class_values}_{subclass_columns}_{subclass_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
        plt.savefig(filename)
        plt.show()

        # Contingency table between clusters and main crop 
        ct_maincrop = pd.crosstab(self.results['Cluster'],self.results['MAIN_CROP'])
        print(ct_maincrop)
        
        # Contingency table between clusters and all multi-cropping (info in LIQ_REPORT)
        ct_multicropping = pd.crosstab(self.results['Cluster'],self.results['LIQ_REPORT'])
        print(ct_multicropping)
        
        # Convert cluster results to a numpy array
        clusters = np.array(self.results['Cluster'])
        
        if self.varColumn=='et' or varColumn=='et_ensemble_mad':
            # Calculate cumulative ET for each UniqueID
            cumulative_et = self.reshaped.cumsum(axis=1)
            
            # Calculate derivative ET for each UniqueID
            derivative_et = np.diff(self.reshaped, axis=1)
    
            # Graphing the cumulative ET
            fig, ax = plt.subplots()
            cluster_counts = [np.sum(self.clusters == cluster_idx) for cluster_idx in range(num_clusters)]
            sorted_clusters = np.argsort(cluster_counts)[::-1]
            if num_clusters <= 5:
                colors = ['red', 'blue', 'green', 'orange', 'purple'][:num_clusters]
            else:
                colors = cm.get_cmap('Spectral', num_clusters)
            for cluster_idx in sorted_clusters:
                cluster_indices = np.where(clusters == cluster_idx)[0]
                for idx in cluster_indices:
                    ax.plot(doy, cumulative_et[idx], color=colors[cluster_idx], linewidth=0.25)
            months = range(1, 13)
            month_labels = [datetime.date(2000, month, 1).strftime('%b') for month in months]
            ax.set_xticks(doy)
            ax.set_xticklabels(month_labels)
            ax.set_title("All Points")
            ax.set_ylabel('Cumulative ET (mm)')
            plt.tight_layout()
            if subclass_values==['**']:
                filename = f"cum_ET_plot_{class_columns}_{class_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
            else: 
                filename = f"cum_ET_plot_{class_columns}_{class_values}_{subclass_columns}_{subclass_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
            plt.savefig(filename)
            plt.show()
            
            # Graphing the derivative ET
            fig, ax1 = plt.subplots()
            cluster_counts = [np.sum(self.clusters == cluster_idx) for cluster_idx in range(num_clusters)]
            sorted_clusters = np.argsort(cluster_counts)[::-1]
            if num_clusters <= 5:
                colors = ['red', 'blue', 'green', 'orange', 'purple'][:num_clusters]
            else:
                colors = cm.get_cmap('Spectral', num_clusters)
            for cluster_idx in sorted_clusters:
                cluster_indices = np.where(clusters == cluster_idx)[0]
                for idx in cluster_indices:
                    if num_clusters <= 5:
                        ax1.plot(doy[1:], derivative_et[idx], color=colors[cluster_idx], linewidth=0.25)
                    else:
                        ax1.plot(doy[1:], derivative_et[idx], color=colors[cluster_idx], linewidth=0.25)
            months = range(1, 13)
            month_labels = [datetime.date(2000, month, 1).strftime('%b') for month in months]
            ax1.set_xticks(doy)
            ax1.set_xticklabels(month_labels)
            ax1.set_title("All Points")
            ax1.set_ylabel('Derivative ET (mm)')
            
            # Add a horizontal line at y=0
            ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
        
            plt.tight_layout()
            if subclass_values==['**']:
                filename = f"derivative_ET_plot_{class_columns}_{class_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
            else: 
                filename = f"derivative_ET_plot_{class_columns}_{class_values}_{subclass_columns}_{subclass_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
            plt.savefig(filename)
            plt.show()
            
            ##### Plotting the NDVI by ET Cluster #####
            
            # Extract UniqueID and Cluster from self.results
            cluster_data = self.results[['UniqueID', 'Cluster']]

            # Map cluster assignments to sentinel DataFrame
            sentinel_clustered = self.sentinel.merge(cluster_data, on='UniqueID')

            # Group the sentinel DataFrame by UniqueID and Cluster
            grouped_sentinel = sentinel_clustered.groupby(['UniqueID', 'Cluster'])

            # Plot a timeseries line for each group
            fig4, ax4 = plt.subplots()
            for (unique_id, cluster), group in grouped_sentinel:
                color = colors[cluster] if num_clusters <= 5 else colors(cluster / num_clusters)  # Using colormap for more clusters
                ax4.plot(np.array(group[self.dateColumn]), np.array(group['NDVI']), color=color, label=f'Cluster {cluster} - ID {unique_id}', linewidth=0.25)
                ax4.set_title('NDVI Time Series for Each UniqueID by Cluster')
                unique_dates = sentinel[dateColumn].unique()
                display_dates = unique_dates[::4]
                ax4.set_xticks(display_dates)
                ax4.set_xticklabels(display_dates, rotation=45)
                ax4.set_xlabel('Date')
                ax4.set_ylabel('NDVI')
            
            plt.tight_layout()
            if subclass_values==['**']:
                filename = f"NDVI_using_ET_clusters_plot_{class_columns}_{class_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
            else: 
                filename = f"NDVI_using_ET_clusters_plot_{class_columns}_{class_values}_{subclass_columns}_{subclass_values}_{startdate}_{enddate}_{varColumn}_{idColumn}_{dateColumn}_number_of_clusters_{num_clusters}.png"
            plt.savefig(filename)
            plt.show()
            print('ran fig4')

Load the data.

In [ ]:
# Load the LandIQ data
landiq = pd.read_csv(r'C:\Users\kdrechsler2\Box\Valley_Water\Tables\LandIQ\i15_Crop_Mapping_2020_Santa_Clara_Table.csv')

# Load the timeseries data
timeseries = pd.read_csv(r'C:\Users\kdrechsler2\Box\Valley_Water\Tables\OpenET\2020_OpenET_zonal_statistics_Ensemble.csv')
#timeseries = pd.read_csv(r'C:\Users\kdrechsler2\Box\Valley_Water\Tables\Sentinel-2\VW_zonal_statistics_2019-2021_mean_uniqueID_copy.csv')

# Load the sentinel timeseries data
sentinel = pd.read_csv(r'C:\Users\kdrechsler2\Box\Valley_Water\Tables\Sentinel-2\VW_zonal_statistics_2019-2021_mean_uniqueID_copy.csv')

# Import the CSV file into a DataFrame
cloudydates = pd.read_csv('unique_dates_cloudy_50percent.csv')
cloudydates['date'] = pd.to_datetime(cloudydates['date'])

Define the parameters for the TimeSeriesAnalysis instance.

In [ ]:
class_columns = ['CLASS1', 'CLASS2', 'CLASS3', 'CLASS4']
subclass_columns = ['SUBCLASS1', 'SUBCLASS2', 'SUBCLASS3', 'SUBCLASS4']
class_values = [' V'] # IMPORTANT: include any required leading spaces
subclass_values = ['**'] # Put ['**'] if blank. REMEMBER leading spaces.
startdate = '2020-01-01' # Start date of the analysis
enddate = '2020-12-31' # End date of the analysis
varColumn = 'et_ensemble_mad' # Column name of the timeseries variable
idColumn = 'UniqueID' # Column name of the unique field ID from LandIQ
dateColumn = 'date' # Column name of the dates
num_clusters = 2 # Number of clusters for the clustering

Filter sentinel data by dates

In [ ]:
sentinel = sentinel[(sentinel['date'] >= startdate) & (sentinel['date'] <= enddate)]

Create an instance of TimeSeriesAnalysis

In [ ]:
analysis = TimeSeriesAnalysis(timeseries, 
                              landiq, 
                              sentinel,
                              class_columns, 
                              subclass_columns, 
                              class_values, 
                              subclass_values, 
                              startdate, 
                              enddate, 
                              varColumn, 
                              idColumn, 
                              dateColumn) 

Run the analysis using the specified parameters  

In [ ]:
analysis.run_analysis(num_clusters)

Mapping and graphing

In [ ]:
# Read the shapefile
shapefile_path = r'C:\Users\kdrechsler2\Box\Valley_Water\Shapefiles\i15_Crop_Mapping_2019_Santa_Clara.shp'
features = gpd.read_file(shapefile_path)
joined = analysis.run_mapping(features)

# Reshape the data for graphing
reshaped = analysis.reshaping(cloudydates)

# Other graphs
analysis.run_graphing()